In [2]:
import numpy as np
import pandas as pd
import pulp
from scipy.spatial import distance_matrix
import itertools
import matplotlib.pyplot as plt

customer_count = 10 #顧客数（id=0はdepot）
vehicle_count = 4 #車両数
vehicle_capacity = 50 #車両容量

np.random.seed(seed=32)

#各顧客のx,y座標をDataFrameとして作成
df = pd.DataFrame({"x":np.random.randint(0,100, customer_count), 
                   "y":np.random.randint(0, 100, customer_count), 
                   "demand":np.random.randint(5, 20, customer_count)})

#id=0はdepotなので，demand=0にする
df.iloc[0].x = 50
df.iloc[0].y = 50
df.iloc[0].demand = 0

#コストとしてノード間の直線距離を求める

#修正前
#cost = pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index).values
#修正後
cost = pd.DataFrame(distance_matrix(df.loc[:,["x","y"]].values, df.loc[:,["x","y"]].values), index=df.index, columns=df.index).values

df.head()

,x,y,demand
0,50,50,0
1,43,4,15
2,5,11,9
3,54,81,7
4,62,3,9


In [3]:

for vehicle_count in range(1,vehicle_count+1):
    # 問題の宣言
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    # 決定変数
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

    # 目的関数
    problem += pulp.lpSum(cost[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) for j in range(customer_count) for i in range (customer_count))

    # 制約
    # (2)式，各顧客の場所に訪れるのは1台の車両で1度である
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(customer_count) for k in range(vehicle_count)) == 1 

    #(3)式, depotから出発して，depotに戻ってくる
    for k in range(vehicle_count):
        # デポを出発した運搬車が必ず 1つの顧客から訪問を開始することを保証する制約条件
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        # 必ず 1 つの顧客から運搬車がデポへ到着すること保証する制約条件
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

    #(4)式, ある顧客の所に来る車両数と出る車両数が同じ
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    #(5)式, 各車両において最大容量を超えない
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 


    #(6)式, 部分巡回路除去制約
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1
    
    #最適化問題を解く
    #最適解が出たら終了
    if problem.solve() == 1:
        print('車両数:', vehicle_count)
        print('目的関数値:', pulp.value(problem.objective))
        print(problem)
        break


車両数: 2
目的関数値: 372.10699852069047
CVRP:
MINIMIZE
46.52956049652737*x0_1,0 + 46.52956049652737*x0_1,1 + 59.54829972383762*x0_2,0 + 59.54829972383762*x0_2,1 + 31.25699921617557*x0_3,0 + 31.25699921617557*x0_3,1 + 48.507731342539614*x0_4,0 + 48.507731342539614*x0_4,1 + 40.85339643163099*x0_5,0 + 40.85339643163099*x0_5,1 + 34.88552708502482*x0_6,0 + 34.88552708502482*x0_6,1 + 25.80697580112788*x0_7,0 + 25.80697580112788*x0_7,1 + 46.87216658103186*x0_8,0 + 46.87216658103186*x0_8,1 + 47.67598976424087*x0_9,0 + 47.67598976424087*x0_9,1 + 46.52956049652737*x1_0,0 + 46.52956049652737*x1_0,1 + 38.63935817272331*x1_2,0 + 38.63935817272331*x1_2,1 + 77.78174593052023*x1_3,0 + 77.78174593052023*x1_3,1 + 19.026297590440446*x1_4,0 + 19.026297590440446*x1_4,1 + 75.8023746329889*x1_5,0 + 75.8023746329889*x1_5,1 + 38.41874542459709*x1_6,0 + 38.41874542459709*x1_6,1 + 41.773197148410844*x1_7,0 + 41.773197148410844*x1_7,1 + 50.15974481593781*x1_8,0 + 50.15974481593781*x1_8,1 + 55.17245689653489*x1_9,0 + 55.